# Modulus

In this notebook we demonstrate the modulo function available within cicada. 

We use a couple fixed operands which we initialize and share. These we then use as operands for the modulo function. Feel free to change these operands. They are a secret operand (secret_a) and a public modulus (b) initialized in their own notebook cell for clarity and convenience. 

We reveal the result and calculate the error based on the result of operating on the underlying values via normal floating point arithmetic and report the results.

Our method here is dependent on our integer division which is approximate and this method is therefore probabilistic. 

In [1]:
import logging

In [2]:
import numpy

In [3]:
import cicada.additive
import cicada.communicator

In [4]:
logging.basicConfig(level=logging.INFO)

In [5]:
b = numpy.array(117)
secret_a = numpy.array(b*1234+31)

In [6]:
@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)

    b_enc = protocol.encoder.encode(numpy.array(b, dtype=object))
    log.info(f"Player {communicator.rank} secret a: {secret_a}", src=0)
    log.info(f"Player {communicator.rank} b: {b}", src=1)
    a_share = protocol.share(src=0, secret=protocol.encoder.encode(numpy.array(secret_a)), shape=())
    residue_share = protocol.modulus_private_public(a_share, b)
    residue = protocol.encoder.decode(protocol.reveal(residue_share))
    log.info(f"Player {communicator.rank} residue {secret_a} % {b}: {residue}\texpected: {secret_a%b}")



In [9]:
main()

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:57688 from tcp://127.0.0.1:57688.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:57688 from tcp://127.0.0.1:57689.
INFO:cicada.communicator.nng:Player 2 rendezvous with tcp://127.0.0.1:57688 from tcp://127.0.0.1:57690.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:root:Player 0 secret a: 144409
INFO:root:Player 1 b: 117
INFO:root:Player 0 residue 144409 % 117: 31.0	expected: 31
INFO:root:Player 1 residue 144409 % 117: 31.0	expected: 31
INFO:root:Player 2 residue 144409 % 117: 31.0	expected: 31
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator freed.
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator freed.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator freed.
INFO:cicada.communicato

[None, None, None]